In [1]:
# pip install pymystem3

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [12]:
# #Create lemmatizer and stopwords list
# mystem = Mystem() 
# russian_stopwords = stopwords.words("russian")
# #Preprocess function
# def preprocess_text(text):
#     tokens = mystem.lemmatize(text.lower())
#     tokens = [token for token in tokens if token not in russian_stopwords\
#               and token != ' '\
#               and token.strip() not in punctuation\
#              ]
#     text = " ".join(tokens)
    
#     return text

In [3]:
# def remove_trash(text):
#     emoj = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"
#                       "]+", re.UNICODE)
    
#     text = re.sub(r'@[А-Яа-я0-9]+', '', text)
#     text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove @
#     text = re.sub(r'#', '', text)  # remove #
#     text = re.sub(r'{', '', text)
#     text = re.sub(r'»', '', text)
#     text = re.sub(r'«', '', text)
#     text = re.sub(r'}', '', text)
#     text = re.sub("[.|/|-|\|,|!]", '', text)  # ~~~~~~~~~~~
#     text = re.sub(r'^а-яА-Я', '', text)
#     text = re.sub('^a-zA-Z', '', text)
#     text = re.sub(r'https?:\/\/\S+', '', text)  # remove hyperlink
#     text = re.sub(r'\n', '', text)
#     text = re.sub(r'———————————————————————————', '', text)
#     text = re.sub(r'•', '', text)
#     text = re.sub(r'\d{4}', '', text)
#     text = re.sub(r'\b\w{2}\b', '', text)


#     text = text.strip()
    
#     return re.sub(emoj, '', text)

In [15]:
df = pd.read_csv('news.csv').drop('Unnamed: 0', axis=1)

In [16]:
# df['news'] = df['news'].apply(lambda x: mystem.lemmatize(x.lower()))
# df['news'] = df['news'].apply(lambda x: remove_trash(x))

In [18]:
train = df[~df['source'].isna()]
test = df[df['source'].isna()]

In [19]:
X = train['news']
y = train['source']
X_val = test['news']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=False)

In [21]:
MAX_WORDS = 100000

MAX_SEQUENCE_LENGTH = 256

In [22]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df['news'])

Wall time: 11 s


In [23]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vetak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
text_train = df['news'].iloc[X_train.index]
text_test = df['news'].iloc[X_test.index]
text_val = df['news'].iloc[X_val.index]

In [25]:
# tokenize.word_index

In [26]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_val_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_val), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_val_sequences.shape, )

(14688, 256) (3672, 256) (1102, 256)
Wall time: 9 s


In [35]:
text_train
# text_train_sequences[0]

0         Официальный аккаунт PlayStation опубликовал т...
1         Китайская компания Mobvoi опубликовала на офи...
2         Практически во всех странах мира введены огра...
3         Депутат Госдумы, единоросс Антон Горелкин вне...
4         Совет директоров «Почты России» проголосовал ...
                               ...                        
14683     Google отмечает 50 лет с момента создания дет...
14684     Стриминговый сервис Netflix в треде в Twitter...
14685     Компания Vivo провела презентацию серии смарт...
14686     В Едином реестре нотификаций о характеристика...
14687     По информации ресурса The Verge , производите...
Name: news, Length: 14688, dtype: object

In [82]:
train_ = pd.DataFrame(text_train_sequences)
test_ = pd.DataFrame(text_test_sequences)
val_ = pd.DataFrame(text_val_sequences)

In [83]:
from sklearn.linear_model import LogisticRegression

In [84]:
model = LogisticRegression()
model.fit(train_.values, y_train.values)

LogisticRegression()

In [85]:
model.predict(test_)

array([1., 1., 0., ..., 1., 0., 1.])

In [86]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [87]:
confusion_matrix(y_test.values, model.predict(test_.values))

array([[ 434,  123,  144],
       [ 126, 2173,   28],
       [ 255,  292,   97]], dtype=int64)

In [88]:
accuracy_score(y_test.values, model.predict(test_.values))

0.7363834422657952

In [89]:
X_ = pd.concat([train_, test_]).values

In [90]:
y_ = train['source'].values

In [91]:
model_ = LogisticRegression()
model.fit(X_, y_)

LogisticRegression()

In [92]:
predictions = pd.DataFrame(model.predict(val_))

In [93]:
predictions.to_csv('sub.csv', index=False)

In [105]:
a = pd.read_csv('sub.csv', header=None)

In [108]:
a[0].value_counts()

1.0    769
0.0    255
2.0     78
Name: 0, dtype: int64